In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime 
from datetime import datetime


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104*",
    security_token="kpa3cq4mF06hRwzJKQvMslTw",
)



In [ ]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

In [ ]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

In [ ]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

In [ ]:
Pacientes_SF.head()

In [ ]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

In [ ]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

In [ ]:
empresas_unidas.head(5)

In [ ]:
Paciente_Programas_SF.head()

In [ ]:
PacientesyEmpresa = pd.merge(
    left=empresas_unidas,
    right=Pacientes_SF,
    how="left",
    left_on="Id Account",
    right_on="Id Account Paciente",
)

In [ ]:
PacientesyEmpresa.head()

In [ ]:
PacientesyEmpresa.shape

In [ ]:

PacientesyEmpresaMetlife = PacientesyEmpresa[(PacientesyEmpresa["Campaña"] == "MetLife")]

In [ ]:
PacientesyEmpresaMetlife.shape

In [ ]:
PacientesyEmpresaMetlife.head()

In [ ]:
PacientesyEmpresaMetlife = PacientesyEmpresaMetlife[
    [
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut",
        "Nombre",
        
    ]
]


In [ ]:
apprix_1 = PacientesyEmpresaMetlife.iloc[:454903,:]
apprix_2 = PacientesyEmpresaMetlife.iloc[454903:,:]

In [ ]:
apprix_1.to_excel("E:/Siempre/04-Comercial/05-MetLife SFTP/18-08-2023/MetLife_SF1.xlsx",index=False)
apprix_2.to_excel("E:/Siempre/04-Comercial/05-MetLife SFTP/18-08-2023/MetLife_SF2.xlsx",index=False)

In [ ]:
tabla_pivote = pd.pivot_table(
    PacientesyEmpresaMetlife,
    
    index=["Empleador","Poliza"],
    values=["Rut"],
    aggfunc=[np.sum,np.count_nonzero],fill_value=0
)

In [ ]:
tabla_pivote

In [ ]:
tabla_pivote.to_excel("F:/martin/MetLife_Resumen2.xlsx",index=True)